In [1]:
import xesmf as xe
import xarray as xr
import numpy as np
import glob
import os
from netCDF4 import Dataset

In [ ]:
def ocean_regrid(expname, varname):
    path_atm = glob.glob(f'/work/Katherine.Turner/tempest-extremes/{expname}/data/*.slp.nc')[0]
    path_atm
    ds_atm = xr.open_dataset(path_atm, use_cftime=True)

    ds_final_grid = xr.Dataset(
        {
            "lat": (["lat"], ds_atm.lat.values, {"units": "degrees_north"}),
            "lon": (["lon"], ds_atm.lon.values, {"units": "degrees_east"}),
        }
    )
    print("Output grid created...")

    files_to_regrid = sorted(glob.glob(f'/work/Katherine.Turner/tempest-extremes/{expname}/data/*.{varname}.nc'))
    
    for i in np.arange(len(files_to_regrid)):
        #check to see if regridded file exists
        file_out = files_to_regrid[i][:-3]+'.regrid.nc'
        file_out_tail = file_out.split('/data/')[-1]
    
        if os.path.isfile(file_out):
            print(f"{file_out_tail} exists. Continuing...")
        
        else:
            print(f"{file_out_tail} does not exist. Regridding...")
        
            ds_in = xr.open_dataset(files_to_regrid[i], use_cftime=True)
            da = ds_in[varname]
            regridder = xe.Regridder(ds_in, ds_final_grid, "bilinear", periodic=True)
            da_out = regridder(da)
            
            ds_out = da_out.to_dataset(name=varname)
            ds_out.to_netcdf(file_out)

In [ ]:
expname = 'odiv-251'
varname = 'MLD_003'

ocean_regrid(expname,varname)

In [7]:
def update_composite_calendar(expname,varname):
    if varname == "t_ref":
        filepath = f"/work/Katherine.Turner/tempest-extremes/{expname}/comp_temp/comp_{varname}.nc"
    elif varname == "MLD_003":
        filepath = f"/work/Katherine.Turner/tempest-extremes/{expname}/comp_MLD/impulse_comp_{varname}_anom.nc" 
    else:
        filepath = f"/work/Katherine.Turner/tempest-extremes/{expname}/comp_{varname}/comp_{varname}_anom.nc"
    ds = Dataset(filepath, "a", format="NETCDF4")

    snap_time = ds.variables["snap_time"]
    snap_time.calendar = "noleap"
    ds.close()

In [8]:
update_composite_calendar("odiv-251", 'MLD_003')

In [3]:
filepath = f"/work/Katherine.Turner/tempest-extremes/odiv-251/comp_hfds/comp_hfds_anom_static.nc"

ds = Dataset(filepath, "a", format="NETCDF4")

snap_time = ds.variables["snap_time"]
snap_time.calendar = "noleap"
ds.close()